### 本课提纲
- 二维格子世界
- 环境模块gym
- 基于Q学习的二维格子世界

在上一课中，我们学习了基础的强化学习知识，并且在一维格子空间中实行了一个强化学习的训练过程，那么我们来把问题搞的复杂一点，我们来看一下二维的格子世界中，应该如何来进行训练。

### 二维格子世界

二维的格子世界和一维格子世界有几个区别：
1. 在二维世界中，机器人不仅可以左右行走，而且还可以前后行走，所以行动的选择有四种。
2. 二维世界的边上都有墙，机器人不能走到外面去。二维格子世界是一个4\*4的大小，所以一共有16个格子可以去。
3. 二维世界中有起始点永远是在左上角，其它15个格子中大部分是安全的，而少数是存在陷阱，而右下解的格子是出口，到达会有奖励。

### 环境模块gym

在python中有一个非常用用的模块，专门用来模拟各种环境，这个模块叫作gym，它是由openAI网站做的模块，我们看一下可以怎么用它。

In [1]:
import gym
import collections
from  gym.envs.toy_text.frozen_lake import  FrozenLakeEnv
import numpy as np

我们先加载一个叫作frozenlake的环境，也就是冰冻之湖，加载了这个环境之后，我们可以通过render函数来观察这个环境的情况。

In [51]:
env = FrozenLakeEnv(is_slippery=False)

In [5]:
env.render()


SFFF
FHFH
FFFH
HFFG


从上面这个简图可以看到，这时有16个字符，排成了4\*4的形式，也就是16个格子，其中S表示了起始点，F表示是冰冻的格子，H表示了有洞的地方，G表示我们的目的地。当机器人开始这个游戏时，永远是从S点开始，当它走到H的地方时，游戏终止，没有回报，当它走到G的地方时，游戏终止，得到回报1。所以我们需要训练机器人，让它学习到一种策略，能绕开冰洞，到达目的地，以得到最大的回报。

下面我们看一下env可以有怎么样的函数，重要的是step函数，它的输入是机器人的动作，输出的主要有三项内容。例如输入的动作是2，那么第一项输出是动作后的状态，也就是新的位置编号，第二项输出是回报，第三项输出是反馈游戏有没有终止。

In [53]:
env.step(2)

(1, 0.0, False, {'prob': 1.0})

在上面我们输入2的动作之后，再观察环境，会发现机器人已经走到了起始点右边的格子上了。

In [54]:
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


机器人的动作有四个选择，分别用数字表示，0表示往左走，1表示往下走，2表示往右走，3表示往上走

如果走到游戏终止，还可以通env.reset来重启游戏。

### 基于Q学习的二维世界

先定义一些参数

In [11]:
GAMMA = 0.5
ALPHA = 0.1
TEST_EPISODES = 30
EPSILON = 0.7

我们把所有的函数都用一个类来归集在一起，其中初始的几个属性比较重要，state是保存当前的位置，values是一个字典，它保存了Q表。feedback_env函数是向环境输入动作，得到环境反馈的功能。有一半的机会随机动作，一半的机会去执行最优动作。最优动作是由best_value_and_action函数负责计算的，它会遍历在某个状态下所有四个动作，看在Q表中哪个动作的Q值最大，那么就选择这个动作。如果所有的Q值都是0，也随机选择一个动作。value_update是用于更新参数的函数，它会将下一状态的Q值算出来，结合即时回报作为修正的目标Q值。play_episode是用于判断某个策略（Q表）的效果是怎么样的。

In [12]:
class Agent:
    def __init__(self):
        self.env = FrozenLakeEnv(is_slippery=False)
        self.state = self.env.reset()
        self.values = collections.defaultdict(float)

    def feedback_env(self):
        old_state = self.state
        if np.random.uniform() > EPSILON:
            action = self.env.action_space.sample()
        else:
            _,action = self.best_value_and_action(old_state)
        new_state, reward, is_done, _ = self.env.step(action)
        self.state = self.env.reset() if is_done else new_state
        return (old_state, action, reward, new_state)


   
    def best_value_and_action(self, state):
        best_value, best_action = None, None
        # 遍历所有的动作
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        if  best_value == 0:  
            best_action = self.env.action_space.sample()
            best_value = self.values[(state, best_action)]
        return best_value, best_action
    

    def value_update(self, s, a, r, next_s):
        best_v, _ = self.best_value_and_action(next_s)
        new_val = r + GAMMA * best_v
        old_val = self.values[(s, a)]
        self.values[(s, a)] = old_val * (1-ALPHA) + new_val * ALPHA

    def play_episode(self, env):  
        total_reward = 0.0
        state = env.reset()
        while True:
            _, action = self.best_value_and_action(state)
            new_state, reward, is_done, _ = env.step(action)
            #self.value_update(self.state, action, reward, new_state)
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

然后我们来用一个循环来计算得到最优策略，目标是用最少的循环次数，来使一次游戏的回报尽可能的高，可以设想，如果是完美的策略，基本上每次游戏都可以达到较高的回报，越接近于1越好，所以我们给出一个阈值，如果平均回报大于0.8，我们就终止实验。整体实验是一个while循环，每次循环中，有两个大的步骤，步骤一是机器人探索格子世界并修改参数，步骤二是用一个内层循环来检查本次修正后的效果，每次循环做20次游戏，然后得到平均回报，存到reward中。

In [13]:
test_env = FrozenLakeEnv(is_slippery=False)
agent = Agent()

iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    s, a, r, next_s = agent.feedback_env()
    agent.value_update(s, a, r, next_s)
    reward = 0.0
    for i in range(TEST_EPISODES):
        reward += agent.play_episode(test_env)
    reward /= TEST_EPISODES
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
        best_reward = reward
    if reward > 0.80:
        print("Solved in %d iterations!" % iter_no)
        break


Best reward updated 0.000 -> 0.033
Best reward updated 0.033 -> 0.100
Best reward updated 0.100 -> 0.133
Best reward updated 0.133 -> 0.167
Best reward updated 0.167 -> 0.200
Best reward updated 0.200 -> 0.233
Best reward updated 0.233 -> 0.267
Best reward updated 0.267 -> 0.300
Best reward updated 0.300 -> 0.400
Best reward updated 0.400 -> 0.433
Best reward updated 0.433 -> 0.533
Best reward updated 0.533 -> 0.667
Best reward updated 0.667 -> 0.700
Best reward updated 0.700 -> 0.733
Best reward updated 0.733 -> 0.800
Best reward updated 0.800 -> 0.867
Solved in 1734 iterations!


我们在下面打印出机器人学到的Q表，也就是游戏策略

In [14]:
agent.values

defaultdict(float,
            {(0, 0): 0.0,
             (0, 1): 0.0,
             (0, 2): 0.0,
             (0, 3): 0.0,
             (4, 0): 0.0,
             (4, 1): 6.250000000000003e-07,
             (4, 2): 0.0,
             (4, 3): 0.0,
             (1, 0): 0.0,
             (1, 1): 0.0,
             (1, 2): 0.0,
             (1, 3): 0.0,
             (2, 0): 0.0,
             (2, 1): 0.0,
             (2, 2): 0.0,
             (2, 3): 0.0,
             (6, 0): 0.0,
             (6, 1): 5.355625000000002e-06,
             (6, 2): 0.0,
             (6, 3): 0.0,
             (8, 0): 0.0,
             (8, 1): 0.0,
             (8, 2): 3.500000000000001e-05,
             (8, 3): 0.0,
             (9, 0): 0.0,
             (9, 1): 0.0039442750000000006,
             (9, 2): 2.8187500000000008e-06,
             (9, 3): 0.0,
             (3, 0): 0.0,
             (3, 1): 0.0,
             (3, 2): 0.0,
             (3, 3): 0.0,
             (13, 0): 0.0,
             (13, 1): 0.0,
    